# Entorno Remoto de Compilación (MambaVision) 🚀

Este notebook se ejecuta enteramente en los servidores de Google Colab conectado desde tu VS Code.
Sigue los pasos secuenciales para descargar tu código desde GitHub y compilar el Kernel Custom de C++ contra los headers de TensorFlow provistos por Google.

In [2]:
import os
import sys

# ==========================================
# CONFIGURACIÓN DE TU REPOSITORIO DE GITHUB
# ==========================================
# IMPORTANTE: Reemplaza esto con la URL real de tu repositorio.
# Si tu repositorio es privado, puedes usar un Personal Access Token (PAT): 
# GITHUB_REPO = "https://TU_TOKEN@github.com/usuario/mambavision_tf_kernel.git"
GITHUB_REPO = "https://github.com/rdromer2/mambavision_tf_kernel.git" # <-- CAMBIAR

REPO_NAME = "mambavision_tf_kernel"
PROJECT_PATH = f"/content/{REPO_NAME}"

# 1. Clonar o actualizar el repositorio
if not os.path.exists(PROJECT_PATH):
    print(f"Clonando el repositorio {GITHUB_REPO}...")
    !git clone {GITHUB_REPO} {PROJECT_PATH}
else:
    print("El repositorio ya existe. Actualizando con git pull...")
    !cd {PROJECT_PATH} && git pull

# 2. Añadir al sys.path por si en el futuro tenemos scripts de python (.py)
sys.path.append(PROJECT_PATH)

# 3. Cambiar nuestro directorio activo de Python y Linux al proyecto
os.chdir(PROJECT_PATH)
print(f"\nDirectorio de trabajo actual: {os.getcwd()}")

Clonando el repositorio https://github.com/rdromer2/mambavision_tf_kernel.git...
Cloning into '/content/mambavision_tf_kernel'...
fatal: could not read Username for 'https://github.com': No such device or address


FileNotFoundError: [Errno 2] No such file or directory: '/content/mambavision_tf_kernel'

## 🛠️ Fase de Compilación C++
Aquí preparamos la máquina virtual (instalando `cmake` si no viniera) y compilamos la librería binaria compartida (`zero_out.so`).

In [ ]:
# Instalar CMake en la VM de Colab
!apt-get update -qq
!apt-get install -y cmake

# Crear carpeta de construcción aislada para no ensuciar la raíz
os.chdir(PROJECT_PATH)
!mkdir -p build
os.chdir('build')

# Configurar CMake referenciando al CMakeLists.txt de la raíz (..)
!cmake ..

In [ ]:
# Ejecutar la compilación en sí. 
# Esto produce el archivo zero_out.so que es lo que TF cargará.
!make

## 🧪 Prueba Inicial del Kernel
Esta celda importa TensorFlow e intenta cargar el `zero_out.so` recién creado. Debería funcionar y mostrar el log correspondiente.

In [ ]:
import tensorflow as tf

try:
    zero_out_module = tf.load_op_library(f'{PROJECT_PATH}/build/zero_out.so')
    # Evaluamos con una matriz de prueba
    print(zero_out_module.zero_out([[1, 2], [3, 4]]))
except Exception as e:
    print("Error al cargar/ejecutar la librería:", e)

## ☁️ (Opcional) Montaje de Google Drive para Datasets/Modelos Pesados
Colab se limpia al cerrarlo. Si vas a entrenar o probar la métrica producida, usa Drive para **LOS DATOS**.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("\n🚀 Google Drive ha sido montado correctamente.")